In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<unknown>:1: SyntaxWarning: invalid escape sequence '\ '


In [31]:
import polars as pl

df = pl.read_csv("../data/test_results/test_output_GoldenChrysus.csv")

In [32]:
df.head()

submission_id,formatted_context,user_comment_ids
str,str,str
"""3gjpz2""","""[ { ""user"": ""SUBJECT"", …","""['ctyrgpd']"""
"""3gpjwq""","""[ { ""user"": ""ari_s_p_e_c…","""['cu11t9e']"""
"""3h7qop""","""[ { ""user"": ""[deleted]"",…","""['cu5o1v4']"""
"""3hnj6j""","""[ { ""user"": ""zer0w0rries…","""['cu98ijr', 'cu98jbq']"""
"""3hqgs5""","""[ { ""user"": ""tropicalrec…","""['cu9nxu1']"""


In [33]:
content = df[3].get_column("formatted_context").item()


In [37]:
from user_embeddings.utils.teacher_prompt import get_teacher_prompt

prompt = get_teacher_prompt(content)
prompt

'\nYou are an Expert Interaction Analyzer and Structure Synthesizer. Your goal is to analyze a conversational context, focusing on the contributions of the participant designated as \'SUBJECT\', and represent the flow of interaction as a nested JSON structure composed of natural language summaries and actions.\n\nInput:\nA JSON structure representing the conversation thread (list of message objects), where the target participant is identified by the username "SUBJECT".\n\nOutput Format Specification:\nGenerate a single JSON object with the following structure:\n{\n  "context": "Natural language string summarizing the initial context before the SUBJECT\'s first action relevant to this structure.",\n  "actions": [\n    // This list contains a mix of strings and nested objects, ordered chronologically.\n    "String representing a single, atomic action/contribution by the SUBJECT.",\n    // ... more action strings if the SUBJECT made multiple points in the same comment ...\n    {\n      "c

In [ ]:
from user_embeddings.utils.get_text_completion import (
    get_openrouter_client,
    get_text_completion,
)

client = get_openrouter_client()

In [46]:
from user_embeddings.utils.teacher_prompt import parse_teacher_prompt_output

parse_teacher_prompt_output("""
```json
{
  "context": "The initial post presented a 'shower thought' about charging modern devices like e-cigarettes and books via USB, which was later attributed to its original author and noted to have been syndicated by Reddit. A reply ('TorinoCobra070') remarked on theRecursive nature of some USB charging devices needing USB ports themselves. This prompted a nested series of replies: a 'Yo Dawg' reference ('robes_pee_air'), comments initiating a discussion about this user's name ('Urgullibl', 'pitchingataint'), an explanation linking it to Robespierre ('Actually_False'), an appreciative reply using the word 'tight' ('pitchingataint'), and finally a comment expressing surprise at the use of 'tight' ('gusherfullofvinegar').",
  "actions": [
    "SUBJECT responds 'Gnarly dude.' to the user expressing surprise at the word 'tight'.",
    "SUBJECT questions the scarcity of USB ports, suggesting they are widely available."
  ]
}
```
""")

{'context': "The initial post presented a 'shower thought' about charging modern devices like e-cigarettes and books via USB, which was later attributed to its original author and noted to have been syndicated by Reddit. A reply ('TorinoCobra070') remarked on theRecursive nature of some USB charging devices needing USB ports themselves. This prompted a nested series of replies: a 'Yo Dawg' reference ('robes_pee_air'), comments initiating a discussion about this user's name ('Urgullibl', 'pitchingataint'), an explanation linking it to Robespierre ('Actually_False'), an appreciative reply using the word 'tight' ('pitchingataint'), and finally a comment expressing surprise at the use of 'tight' ('gusherfullofvinegar').",
 'actions': ["SUBJECT responds 'Gnarly dude.' to the user expressing surprise at the word 'tight'.",
  'SUBJECT questions the scarcity of USB ports, suggesting they are widely available.']}

In [40]:
result = await get_text_completion("google/gemma-3-27b-it", prompt)
print(result)

```json
{
  "context": "A user ('zer0w0rries') shared a 'showerthought' about the absurdity of needing a USB port to charge a cigarette while simultaneously needing one to charge a book, noting the original source of the thought on Twitter and subsequent unauthorized use of the post by Reddit on Instagram and Facebook. A conversation then evolved about USB chargers, user names, and historical figures (Robespierre).",
  "actions": [
    "SUBJECT expresses agreement with a previous comment using the slang term 'Gnarly dude'.",
    "SUBJECT rhetorically questions the scarcity of USB ports, implying that having access to them is commonplace."
  ]
}
```
